# 😦 Experiencing Technical Difficulties! 😦

Since Wine.com deprecated their API as of Sept-17, this code no longer works. However it still useful as documentation for how I obtained the dataset in September-2015. Hindsight is 20/20, and as this was my project for General Assembly, and first foray into coding with Python, I would probably have taken some steps to streamline the API data wrangling. 

### Full write-up and project description will be given in other notebooks.

In [1]:
#Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns
%matplotlib inline

#tools to help with JSON
import json
from pandas.io.json import json_normalize
#pprint - pretty print - preserves formatting
from pprint import pprint

#urllib / requests - used commonly for apis
import requests
import urllib2

In [ ]:
cab_df = pd.read_csv('./cablist.csv')

### Extract data from Wine.com API & Build baseline dataframe using urlib -- hacked with help from GA staff!

In [1]:
#open data from JSON format
request=urllib2.Request('http://services.wine.com/api/beta2/service.svc/json/catalog?search=cabernet+sauvignon&offset=4001&size=100&apikey=a290c490a29709b702c1c79174821e8f')
response = urllib2.urlopen(request)
elevations = response.read()
data = json.loads(elevations)

NameError: name 'urllib2' is not defined

In [3]:
#extract product information- dictionary of lists (with dictionaries)
wine = data['Products']['List']

### This is what the data looks like... Lots of nested dictionaries & lists

In [4]:
wine[0]

{u'Appellation': {u'Id': 9058,
  u'Name': u'Barossa Valley',
  u'Region': {u'Area': None,
   u'Id': 108,
   u'Name': u'Australia',
   u'Url': u'http://www.wine.com/v6/Australia/wine/list.aspx?N=7155+108'},
  u'Url': u'http://www.wine.com/v6/Barossa-Valley/wine/list.aspx?N=7155+108+9058'},
 u'Community': {u'Reviews': {u'HighestScore': 0,
   u'List': [],
   u'Url': u'http://www.wine.com/v6/Branson-Coach-House-Coach-House-Block-Cabernet-Sauvignon-2004/wine/89049/Detail.aspx?pageType=reviews'},
  u'Url': u'http://www.wine.com/v6/Branson-Coach-House-Coach-House-Block-Cabernet-Sauvignon-2004/wine/89049/Detail.aspx'},
 u'Description': u'',
 u'GeoLocation': {u'Latitude': -360, u'Longitude': -360, u'Url': u''},
 u'Id': 89049,
 u'Labels': [{u'Id': u'89049m',
   u'Name': u'thumbnail',
   u'Url': u'http://cache.wine.com/labels/89049m.jpg'}],
 u'Name': u'Branson Coach House Coach House Block Cabernet Sauvignon 2004',
 u'PriceMax': 99.0,
 u'PriceMin': 95.0,
 u'PriceRetail': 99.0,
 u'ProductAttribute

### I'll need to use a list comprehension to extract my baseline data fields

In [5]:
#List comprehension to create baseline data
#'Name' -4 is the vintage
p = [[w['Id'], w['ProductAttributes'],w['Type'],w['Name'],w['Name'][-4:],w['Appellation']['Name'],w['Appellation']['Region']['Name'],w['Varietal']['Name'], w['Vineyard']['Name'], w['Ratings']['HighestScore'],w['PriceRetail'] ]for w in wine]

In [6]:
#Filter baseline data for wine only, and only those records with product attributes listed
v = []
for i in p:
    if len(i[1]) > 0 and i[2]=='Wine':
        v.append(i)


In [7]:
len(v)

99

### Here's what the data now looks like. There's still a dictionary for the Product Attribute field, so going to run another list comprehension

In [8]:
v[0]

[89049,
 [{u'Id': 36,
   u'ImageUrl': u'http://cache.wine.com/assets/glo_icon_collectable_big.gif',
   u'Name': u'Collectible Wines',
   u'Url': u'http://www.wine.com/v6/Collectible-Wines/wine/list.aspx?N=7155+36'}],
 u'Wine',
 u'Branson Coach House Coach House Block Cabernet Sauvignon 2004',
 u'2004',
 u'Barossa Valley',
 u'Australia',
 u'Cabernet Sauvignon',
 u'Branson Coach House',
 98,
 99.0]

In [9]:
#Second list comprehension to extract product attribute from dictionary
wl = [[i[0], i[1][0]['Name'], i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10]] for i in v]

In [10]:
#Only run this to initialize wl when pulling initial data
cab_list = wl

In [807]:
#appends results of api call to existing list
#cab_list.extend(wl)

In [11]:
len(cab_list)

99

### Success! Now the data is in a format that I can work with. 

In [12]:
cab_list[0]

[89049,
 u'Collectible Wines',
 u'Branson Coach House Coach House Block Cabernet Sauvignon 2004',
 u'2004',
 u'Barossa Valley',
 u'Australia',
 u'Cabernet Sauvignon',
 u'Branson Coach House',
 98,
 99.0]

### The next 4 cells were used as part of the process of building the dataset. The API call is limited to 100 records at a time, so I didn't pull them all at once. I built the file up over time -- lots of time -- and these cells are part of the merge process and data inspection.

In [841]:
#create dataframe from list of results, in preparation to merge with existing dataframe
temp_df = pd.DataFrame(cab_list)[300:1000]
temp_df.columns = ['Id', 'Attributes', 'Name', 'Vintage', 'Appellation', 'Region','WineType', 'Vineyard', 'RatingScore', 'PriceRetail']

In [843]:
#merging dataframes: new + existing
frames = [cab_df, temp_df]
cabernet_df = pd.concat(frames)

In [844]:
#Let's inspect the dataset
cabernet_df.describe()

,Id,RatingScore,PriceRetail
count,2770.000000,2770.000000,2770.000000
mean,112546.000722,93.058123,135.094065
std,25897.287147,3.066592,252.947503
min,316.000000,0.000000,7.990000
25%,103055.250000,91.000000,41.990000
50%,117722.000000,93.000000,77.990000
75%,129950.000000,95.000000,139.000000
max,148089.000000,100.000000,6000.000000


In [849]:
#More inspection
cabernet_df['RatingScore'].value_counts()

90     522
92     479
93     404
94     383
95     338
96     200
89     112
97      97
98      84
91      82
100     36
99      32
0        1
dtype: int64

### That's a wrap -- time to export and play with some data!

In [845]:
#export dataframe to csv, baseline file
cabernet_df.to_csv('cablist.csv', index=False, header=True)

#Create categorical variables 

In [13]:
#read in dataset
cablist_df = pd.read_csv('./cablist.csv')

###The first step was to create a categorical variable based on if the wine's label text conveyed any information not already captured in the appellation, wine type, producer, etc. This should be marketing words that might convey a particular price point, i.e. 'Reserve'

In [14]:
#First step: combine all text fields into a single list
wine_names = cablist_df['Name'].tolist()
appellation = cablist_df['Appellation'].tolist()
wine_type = cablist_df['WineType'].tolist()
vineyard = cablist_df['Vineyard'].tolist()
vintage = cablist_df['Vintage'].tolist()


cabs = zip(wine_names, appellation, wine_type, vineyard, vintage)

###Let's look at this record as an example. 'Reserve' should be the one word that is not present in the appellation, type of wine, producer or vintage.

In [15]:
cabs[67]

('Robert Mondavi Reserve Cabernet Sauvignon 2010',
 'Napa Valley',
 'Cabernet Sauvignon',
 'Robert Mondavi Winery',
 2010)

###Breaking each field into lists for comparison

In [16]:
#Create a list of lists by splitting each text field into individual words for comparison
words = [[w[0].split(), w[1].split(), w[2].split(), w[3].split()] for w in cabs]

In [17]:
words[67]

[['Robert', 'Mondavi', 'Reserve', 'Cabernet', 'Sauvignon', '2010'],
 ['Napa', 'Valley'],
 ['Cabernet', 'Sauvignon'],
 ['Robert', 'Mondavi', 'Winery']]

###Create the list of unqiue words for each wine. Here we can see that 'Reserve' has been added to the list.

In [18]:
#Create list of unique words in label text, not conveyed in vineyard appellation or wine type
#for each individual wine.
unq_words = []
for i in words:
    i[0].pop()
    label = i[0]
    appellation = i[1]
    wine_type = i[2]
    vineyard = i[3]
    pass1 = list(set(label)-set(appellation)-set(wine_type)-set(vineyard))
    unq_words.append(pass1)

In [11]:
unq_words[67]

['Reserve']

###Because some wines had more than one unique word, I looked at the top 5 words of the first and second identified unique words for a given wine. I found that Vineyard was one of the words in the first list, and decided to ignore it- it felt too common and would not be a signifier of price-point to a consumer

In [20]:
#Create dataframe of top 9 unique words from first 2 words for each wine. Only selecting 9
#as the first word was Vineyard- seemed too common
words_df = pd.DataFrame(unq_words)
w1 = list(words_df[0].value_counts()[1:5].index.values)
w2 = list(words_df[1].value_counts()[:5].index.values)
unq_w = w1 + w2

In [21]:
unq_w

['Reserve',
 'Mountain',
 'Liter',
 'Estate',
 'Magnum)',
 'Estate',
 'Reserve',
 'Howell',
 '(375ML']

###Now that I have a list of unique words identified, I want to compare them with the label text of each wine and create an indicator if the label text contains any of the unique words.

In [22]:
#Create list of words in each label
label_words = cablist_df['Name'].str.split(' ').tolist()

In [23]:
label_words[67]

['Robert', 'Mondavi', 'Reserve', 'Cabernet', 'Sauvignon', '2010']

In [24]:
#Create indicator to denote intersection of identified unique words and label words
#Using the Robert Mondavi example, indicator would = 1 as Reserve is in the unique word list
#and also present in the label
words = []
for i in label_words:
    if len(list(set(unq_w) & set(i))) > 0:
        words.append(1)
    else:
        words.append(0)

In [25]:
#Create words dataframe for merging with baseline data
words_df = pd.DataFrame(words)
words_df.columns = ['UnqWordInd']

In [26]:
sum(words)

742

In [27]:
#Merge dataframes
cab_df2 = pd.concat([cablist_df, words_df], axis=1)

In [28]:
len(cab_df2)

2770

In [29]:
cab_df2.head()

,Id,Attributes,Name,Vintage,Appellation,Region,WineType,Vineyard,RatingScore,PriceRetail,UnqWordInd
0,140247,Big &amp; Bold,Screaming Eagle Cabernet Sauvignon 2012,2012,Napa Valley,California,Cabernet Sauvignon,Screaming Eagle,100,2499,0
1,147150,Collectible Wines,Penfolds Kalimna Block 42 Cabernet Sauvignon 2004,2004,Barossa Valley,Australia,Cabernet Sauvignon,Penfolds Wines,100,739,0
2,82919,Collectible Wines,Quilceda Creek Cabernet Sauvignon 2002,2002,Columbia Valley,Washington,Cabernet Sauvignon,Quilceda Creek Vintners,100,269,0
3,139436,Big &amp; Bold,Schrader Old Sparky Cabernet Sauvignon (1.5 Li...,2005,Napa Valley,California,Cabernet Sauvignon,Schrader Cellars,100,1349,1
4,87182,Collectible Wines,Quilceda Creek Cabernet Sauvignon 2003,2003,Columbia Valley,Washington,Cabernet Sauvignon,Quilceda Creek Vintners,100,289,0


In [22]:
#Export list to csv for backup- it took me awhile to get to this point!
cab_df2.to_csv('cablist2.csv', index=False, header=True)

###Now I'm going to create dummy variables for the Region and Attributes fields, and merge with the existing dataframe

In [30]:
cab_df2['Region'].value_counts()

California       2084
Australia         230
South America     204
Washington        202
South Africa       21
Italy              15
Spain              10
Israel              4
dtype: int64

In [31]:
#create dummy variables for attributes variable
attribute_dummies = pd.get_dummies(cab_df2.Attributes, prefix='Attribute').ix[:, 1:]
region_dummies = pd.get_dummies(cab_df2.Region, prefix='Region').ix[:, 1:]

In [32]:
#Merge dummies with existing dataframe
cab_df2 = pd.concat([cab_df2, attribute_dummies], axis=1)

In [33]:
cab_df2 = pd.concat([cab_df2, region_dummies], axis=1)

In [34]:
cab_df2.head()

,Id,Attributes,Name,Vintage,Appellation,Region,WineType,Vineyard,RatingScore,PriceRetail,...,Attribute_Rich &amp; Creamy,Attribute_Screw Cap Wines,Attribute_Smooth &amp; Supple,Region_California,Region_Israel,Region_Italy,Region_South Africa,Region_South America,Region_Spain,Region_Washington
0,140247,Big &amp; Bold,Screaming Eagle Cabernet Sauvignon 2012,2012,Napa Valley,California,Cabernet Sauvignon,Screaming Eagle,100,2499,...,0,0,0,1,0,0,0,0,0,0
1,147150,Collectible Wines,Penfolds Kalimna Block 42 Cabernet Sauvignon 2004,2004,Barossa Valley,Australia,Cabernet Sauvignon,Penfolds Wines,100,739,...,0,0,0,0,0,0,0,0,0,0
2,82919,Collectible Wines,Quilceda Creek Cabernet Sauvignon 2002,2002,Columbia Valley,Washington,Cabernet Sauvignon,Quilceda Creek Vintners,100,269,...,0,0,0,0,0,0,0,0,0,1
3,139436,Big &amp; Bold,Schrader Old Sparky Cabernet Sauvignon (1.5 Li...,2005,Napa Valley,California,Cabernet Sauvignon,Schrader Cellars,100,1349,...,0,0,0,1,0,0,0,0,0,0
4,87182,Collectible Wines,Quilceda Creek Cabernet Sauvignon 2003,2003,Columbia Valley,Washington,Cabernet Sauvignon,Quilceda Creek Vintners,100,289,...,0,0,0,0,0,0,0,0,0,1
